In [1]:
# load libraries and scripts
using PyPlot
using PyCall
using LaTeXStrings
using FFTW
include("../scripts/tools.jl")
include("../scripts/get_grid.jl")
include("../scripts/get_params.jl")
include("../scripts/stable.jl")
include("../scripts/dynamics.jl")
include("../scripts/disturbances.jl")
include("../scripts/ps_analysis.jl");

LoadError: InitError: PyError (PyImport_ImportModule

The Python package matplotlib could not be imported by pyimport. Usually this means
that you did not install matplotlib in the Python version being used by PyCall.

PyCall is currently configured to use the Python version at:

/usr/bin/python3

and you should use whatever mechanism you usually use (apt-get, pip, conda,
etcetera) to install the Python package containing the matplotlib module.

One alternative is to re-configure PyCall to use a different Python
version on your system: set ENV["PYTHON"] to the path/name of the python
executable you want to use, run Pkg.build("PyCall"), and re-launch Julia.

Another alternative is to configure PyCall to use a Julia-specific Python
distribution via the Conda.jl package (which installs a private Anaconda
Python distribution), which has the advantage that packages can be installed
and kept up-to-date via Julia.  As explained in the PyCall documentation,
set ENV["PYTHON"]="", run Pkg.build(

In [2]:
# load borders
border, scale_factor = import_border("../data/borders/border.json")

dx = 0.005
mesh = get_grid(border, dx); # renamed to be able to use the grid function for plotting

LoadError: UndefVarError: import_border not defined

In [3]:
# compute or load grid parameters
contmod = get_params(mesh, scale_factor, "../data/disc/pantagruel.h5", Niter=10,
    dmax = 2*dx, patch=1900.0, bmin=52.0, tau=5.0e-6);

LoadError: UndefVarError: dx not defined

In [4]:
# get stable solution
compute_stable_sol!(contmod, interval = 10000, Niter = 250000, tol = 1e-7);
dm = load_discrete_model("../data/disc/pantagruel.h5", scale_factor);
th_disc = get_discrete_values(contmod.coord[contmod.isgrid,:], dm.coord, contmod.th[contmod.isgrid]);

LoadError: UndefVarError: compute_stable_sol! not defined

In [5]:
# Second model for fourier filtering
contmod2 = get_params(mesh, scale_factor, "../data/disc/pantagruel.h5", Niter=10,
    dmax = 2*dx, patch=1900.0, bmin=52.0, tau=5.0e-7);

LoadError: UndefVarError: dx not defined

In [6]:
# Get frequencies
freq = zeros(contmod2.Ny, contmod2.Nx)
freqx = fftfreq(contmod2.Nx, dx)
freqy = fftfreq(contmod2.Ny, dx)
for i=1:contmod2.Ny
    for j=1:contmod2.Nx
        freq[i, j] = sqrt(freqy[i]^2 + freqx[j]^2)
    end
end

LoadError: UndefVarError: contmod2 not defined

In [7]:
# Create filter mask
filter = freq .<= .2 * maximum(freq);

LoadError: UndefVarError: freq not defined

In [8]:
# Filter m
mfft = fft(reshape(contmod2.m, contmod2.Ny, contmod2.Nx))
mfft[.!filter] .= 0.0
mifft = real.(ifft(mfft))
mifft[.!contmod2.isgrid] .= 0.0
contmod2.m = vec(mifft);

LoadError: UndefVarError: contmod2 not defined

In [9]:
# Filter d
dfft = fft(reshape(contmod2.d, contmod2.Ny, contmod2.Nx))
dfft[.!filter] .= 0.0
difft = real.(ifft(dfft))
difft[.!contmod2.isgrid] .= 0.0
contmod2.d = vec(difft);

LoadError: UndefVarError: contmod2 not defined

In [10]:
# Filter bx
bxfft = fft(reshape(contmod2.bx, contmod2.Ny, contmod2.Nx))
bxfft[.!filter] .= 0.0
bxifft = real.(ifft(bxfft))
bxifft[.!contmod2.isgrid] .= 0.0
contmod2.bx = vec(bxifft);

LoadError: UndefVarError: contmod2 not defined

In [11]:
# Filter by
byfft = fft(reshape(contmod2.by, contmod2.Ny, contmod2.Nx))
byfft[.!filter] .= 0.0
byifft = real.(ifft(byfft))
byifft[.!contmod2.isgrid] .= 0.0
contmod2.by = vec(byifft);

LoadError: UndefVarError: contmod2 not defined

In [12]:
# Filter p
ptemp = zeros(contmod2.Ny * contmod2.Nx)
ptemp[contmod2.isgrid] = contmod2.p
pfft = fft(reshape(ptemp, contmod2.Ny, contmod2.Nx))
pfft[.!filter] .= 0.0
pifft = real.(ifft(pfft))
pifft[.!contmod2.isgrid] .= 0.0
ptemp2 = vec(pifft)[contmod2.isgrid]
ptemp2 .-= sum(ptemp2) / sum(contmod2.isgrid)
contmod2.p = ptemp2;

LoadError: UndefVarError: contmod2 not defined

In [13]:
# get stable solution for filtered model
compute_stable_sol!(contmod2, interval = 10000, Niter = 250000, tol = 1e-7);

LoadError: UndefVarError: compute_stable_sol! not defined

In [14]:
CMAP = :inferno
msize = 0.5
fig = plt.figure(figsize=(8*2/3,10))
gs = fig.add_gridspec(5,2)
# Turn off axis lines and ticks of the big subplot
wholefig = fig.add_subplot(py"$(gs)[:,:]")
wholefig.spines["top"].set_visible(false)
wholefig.spines["right"].set_visible(false)
wholefig.spines["bottom"].set_visible(false)
wholefig.spines["left"].set_visible(false)
wholefig.tick_params(labelcolor="white", top=false, bottom=false, left=false, right=false)
p11 = fig.add_subplot(gs[1,1])
p11.scatter(contmod.coord[contmod.isinside,2], contmod.coord[contmod.isinside,1], c=contmod.p[contmod.isinside[contmod.isgrid]], s=msize, cmap=CMAP)
p11.text(0.05, 0.6, "\$p(\\mathbf{r})\$", transform=p11.transAxes, rotation=90, fontsize=15)
p11.plot(border[:,1], border[:,2], "k", lw=1)
p11.set_aspect("equal")
p11.axis("off")
p12 = fig.add_subplot(gs[2,1])
p12.scatter(contmod.coord[contmod.isgrid,2], contmod.coord[contmod.isgrid,1], c=contmod.m[contmod.isgrid], s=msize, cmap=CMAP)
p12.text(0.05, 0.6, "\$m(\\mathbf{r})\$", transform=p12.transAxes, rotation=90, fontsize=15)
p12.plot(border[:,1], border[:,2], "k", lw=1)
p12.set_aspect("equal")
p12.axis("off")
p13 = fig.add_subplot(gs[3,1])
p13.scatter(contmod.coord[contmod.isgrid,2], contmod.coord[contmod.isgrid,1], c=contmod.d[contmod.isgrid], s=msize, cmap=CMAP)
p13.text(0.05, 0.6, "\$d(\\mathbf{r})\$", transform=p13.transAxes, rotation=90, fontsize=15)
p13.plot(border[:,1], border[:,2], "k", lw=1)
p13.set_aspect("equal")
p13.axis("off")
p14 = fig.add_subplot(gs[4,1])
p14.scatter(contmod.coord[contmod.isgrid,2], contmod.coord[contmod.isgrid,1], c=contmod.bx[contmod.isgrid], s=msize, cmap=CMAP)
p14.text(0.05, 0.6, "\$b_x(\\mathbf{r})\$", transform=p14.transAxes, rotation=90, fontsize=15)
p14.plot(border[:,1], border[:,2], "k", lw=1)
p14.set_aspect("equal")
p14.axis("off")
p15 = fig.add_subplot(gs[5,1])
p15.scatter(contmod.coord[contmod.isgrid,2], contmod.coord[contmod.isgrid,1], c=contmod.by[contmod.isgrid], s=msize, cmap=CMAP)
p15.text(0.05, 0.6, "\$b_y(\\mathbf{r})\$", transform=p15.transAxes, rotation=90, fontsize=15)
p15.plot(border[:,1], border[:,2], "k", lw=1)
p15.set_aspect("equal")
p15.axis("off")
p21 = fig.add_subplot(gs[1,2])
p21.scatter(contmod2.coord[contmod2.isinside,2], contmod2.coord[contmod2.isinside,1], c=contmod2.p[contmod2.isinside[contmod2.isgrid]], s=msize, cmap=CMAP)
p21.plot(border[:,1], border[:,2], "k", lw=1)
p21.set_aspect("equal")
p21.axis("off")
p22 = fig.add_subplot(gs[2,2])
p22.scatter(contmod2.coord[contmod2.isinside,2], contmod2.coord[contmod2.isinside,1], c=contmod2.m[contmod2.isinside], s=msize, cmap=CMAP)
p22.plot(border[:,1], border[:,2], "k", lw=1)
p22.set_aspect("equal")
p22.axis("off")
p23 = fig.add_subplot(gs[3,2])
p23.scatter(contmod2.coord[contmod2.isinside,2], contmod2.coord[contmod2.isinside,1], c=contmod2.d[contmod2.isinside], s=msize, cmap=CMAP)
p23.plot(border[:,1], border[:,2], "k", lw=1)
p23.set_aspect("equal")
p23.axis("off")
p24 = fig.add_subplot(gs[4,2])
p24.scatter(contmod2.coord[contmod2.isinside,2], contmod2.coord[contmod2.isinside,1], c=contmod2.bx[contmod2.isinside], s=msize, cmap=CMAP)
p24.plot(border[:,1], border[:,2], "k", lw=1)
p24.set_aspect("equal")
p24.axis("off")
p25 = fig.add_subplot(gs[5,2])
p25.scatter(contmod2.coord[contmod2.isinside,2], contmod2.coord[contmod2.isinside,1], c=contmod2.by[contmod2.isinside], s=msize, cmap=CMAP)
p25.plot(border[:,1], border[:,2], "k", lw=1)
p25.set_aspect("equal")
p25.axis("off")
wholefig.annotate("", xy=(0.75, -0.01), xycoords="axes fraction", xytext=(0.25, -0.01),
            arrowprops=Dict([("arrowstyle","->"), ("color","k")]))
wholefig.text(0.5, -0.03, "Fourier Filtering", transform=wholefig.transAxes, fontsize=15, ha="center")
plt.subplots_adjust(left=0.0, right=1.0, top=1.0, bottom=0.0, wspace=0.05, hspace=0.05)
# fig.tight_layout()
fig.savefig("../figures/param_fourier.pdf", bbox_inches="tight")


LoadError: UndefVarError: plt not defined

In [15]:
msize = 2
clim = extrema([contmod.th; contmod2.th])
fig, axs = plt.subplots(1,3, figsize=(8,2.5), gridspec_kw=py"{\"width_ratios\":[1,1,0.05]}")
axs[1].scatter(contmod.coord[contmod.isinside,2], contmod.coord[contmod.isinside,1], c=contmod.th[contmod.isinside], s=msize, cmap=CMAP, vmin=clim[1], vmax=clim[2])
axs[2].scatter(contmod2.coord[contmod2.isinside,2], contmod2.coord[contmod2.isinside,1], c=contmod2.th[contmod2.isinside], s=msize, cmap=CMAP, vmin=clim[1], vmax=clim[2])
axs[1].set_aspect("equal")
axs[2].set_aspect("equal")
axs[1].axis("off")
axs[2].axis("off")
axs[1].text(0.1, 0.9, "(a)", transform=axs[1].transAxes)
axs[2].text(0.1, 0.9, "(b)", transform=axs[2].transAxes)
fig.colorbar(axs[2].collections[1], cax=axs[3])
plt.subplots_adjust(left=0.0, right=0.9, top=1.0, bottom=0.0, wspace=0.05, hspace=0.05)
plt.savefig("../figures/thetas_fourier.pdf")

LoadError: UndefVarError: contmod not defined